In [1]:
import os
MODEL = "gemma2:latest"

In [2]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
model=Ollama(model=MODEL)
embeddings =  OllamaEmbeddings()

C:\Users\Kodit\AppData\Local\Temp\ipykernel_40652\4188133549.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model=Ollama(model=MODEL)
C:\Users\Kodit\AppData\Local\Temp\ipykernel_40652\4188133549.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings =  OllamaEmbeddings()


In [3]:
model.invoke("who are u?")

"I am Gemma, an open-weights AI assistant. I'm a large language model trained by Google DeepMind. My purpose is to help users by understanding and responding to their requests in a helpful, informative, and comprehensive way.\n\nHere are some key things to know about me:\n\n* **Open-Weights:** My weights are publicly available, meaning anyone can access and study them.\n* **Text-Only:** I can only communicate through text. I can't generate images, sound, or videos.\n* **Limited Knowledge:** I don't have access to real-time information or the internet. My knowledge is based on the data I was trained on, which has a cutoff point.\n* **Created by the Gemma Team:** I was developed by a team of engineers and researchers at Google DeepMind.\n\nI'm always learning and improving, and I'm excited to see how people use me to explore new ideas and accomplish their goals."

In [4]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
chain=model|parser #The chain takes the model, and pipes it into the input of the next model, basically converting the output to a string.
chain.invoke("Tell me a joke")

"Why don't scientists trust atoms?\n\nBecause they make up everything! 🧪😄  \n\n\nLet me know if you want to hear another one! 😊  \n"

In [5]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("HO.pdf")
pages=loader.load_and_split()
pages

[Document(metadata={'source': 'HO.pdf', 'page': 0, 'page_label': '1'}, page_content="BITS PILANI, DUBAI CAMPUS \nACADEMIC – UNDERGRADUATE STUDIES DIVISION \nSECOND SEMESTER 2022 – 2023 \n \nCourse Handout (Part – II) \n            Date: 02.02.2023 \nIn addition to Part I (General Handout for all courses appended to the Time Table) this portion further specific \ndetails regarding the course.                                                                                                     \n \nCourse No   : HSS F365  (3 0 3)  \nCourse Title   : SCIENCE OF SUSTAINABLE HAPPINESS \nInstructor-in-charge  : Dr. Shazi Shah Jabeen \nInstructors   : Dr. Shazi Shah Jabeen \n \nScope and objective of the course: \nThis course aims at bringing about an understanding of what happiness is; why it matters to everyone ; key \npsychological, social, and biological factors that relate to happiness; and the relationship between happiness, \nhuman connection, and prosocial qualities, such as compassion,

In [6]:
from langchain.prompts import PromptTemplate
template="""
Answer the question based on the context below and be accurate. If you cant, 
answer the question, reply "I dont know".

Context:{context}

Question:{question}"""

prompt= PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below and be accurate. If you cant, 
answer the question, reply "I dont know".

Context:Here is some context

Question:Here is a question


In [7]:
chain = prompt | model | parser 

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)
retriever=vectorstore.as_retriever()

In [11]:
from operator import itemgetter
chain=({
    "context":itemgetter("question")| retriever, #SO the question is also passed to the retriever,
    "question":itemgetter("question")
}
|prompt|model|parser)
question=input("Enter the question from the file required:")

print(f"Answer:{chain.invoke({'question':question})}")

Answer:The IC's name is Dr. Shazi Shah Jabeen.  

